# Setting the Stage

JAX is a numerical computing library that is designed to be composable, modular, and extensible. It is built on top of XLA, which is a compiler that can take a function and compile it into a highly optimized version that can run on a variety of hardware. JAX is designed to be a drop-in replacement for NumPy, but it also has a number of features that make it more powerful and flexible than NumPy.

JAX is built from three main components:
- `jax.vmap`: A function that takes a Python function and vectorizes it.
- `jax.jit`: A function that creates a compiled and optimized version of a function.
- `jax.grad`: A function that automatically differentiates another Python function.



## Installation

Installing JAX is typically as simple as running:

```bash
$ pip install jax
```

For a more detailed installation guide, see the [JAX documentation](https://jax.readthedocs.io/en/latest/installation.html).

## Using JAX as a Drop-In Replacement for NumPy

JAX includes most of the functionality you might be familiar with from NumPy, rewritten to utilize XLA instructions and to be compatible with JAX's other features. This is provided through the `jax.numpy` module, typically aliased as `jnp`. 

In [1]:
import jax.numpy as jnp

JAX can then be utilized in much the same way as NumPy. Taking the simple example of a matrix-vector product of
```{math}
:label: matvec
y = Ax
```
for instance,
```{math}
\begin{bmatrix}
    0 & 1 \\
    2 & 3 \\
    4 & 5
\end{bmatrix}
\begin{bmatrix}
    1 \\ 1
\end{bmatrix}
=
\begin{bmatrix}
    1 \\ 5 \\ 9
\end{bmatrix}.
```
This is implemented in JAX as simple as

In [2]:
A = jnp.arange(6).reshape(3, 2)
x = jnp.ones(2)

print(A @ x)

[1. 5. 9.]


## Vectorization through `vmap`

The beauty of JAX then comes from the concept of _composing_ functions, allowing complex operations to be natively written in a fashion closely resembeling their mathematical counterparts. Continuing with the matrix-vector product example, consider the case where instead of a single vector $x \in \mathbb{R}^d$, we have a set of $N$ vectors $X$. We are then interested in computing
```{math}
    y_i = Ax_{i} \quad \text{for } i  = 1, 2, \ldots, N,
```
for a matrix $A \in \mathbb{R}^{m \times d}$.

In NumPy, we would typically store these vectors in a matrix $X \in \mathbb{R}^{N \times d}$ and compute the matrix-vector product $AX^T$. We would then be left with a matrix $Y \in \mathbb{R}^{m \times N}$, where each column $Y_i$ corresponds to the result of the matrix-vector product $Ax_i$.

Keeping track of these dimensionalities can quickly become cumbersome, especially when performed in sequence. A band-aid to this would be to actually compute $(AX^T)^T = XA^T$, however we are then further removed from the mathematical notation.

In [3]:
import numpy as np

rng = np.random.default_rng(2002)
A = np.arange(6).reshape(3, 2)
X = rng.integers(-3, 3, size=(4, 2))


def apply_matrix(A: np.ndarray, X: np.ndarray) -> np.ndarray:
    return X @ A.T


Y = apply_matrix(A, X)
print(Y)

[[  2  10  18]
 [  2   0  -2]
 [  0   4   8]
 [ -3 -13 -23]]


An alternative, seemingly little known, approach in NumPy would be to use `np.einsum`, which allows for Einstein summation notation. This is a powerful tool, but can be difficult to read and write, especially for more complex operations.

In [4]:
def apply_matrix_einsum(A: np.ndarray, X: np.ndarray) -> np.ndarray:
    return np.einsum("ij,Nj->Ni", A, X)


Y_einsum = apply_matrix_einsum(A, X)
print(np.allclose(Y, Y_einsum))

True


In JAX, we can instead use `jax.vmap` to vectorize the matrix-vector product, allowing us to compute the matrix-vector product for each vector in $X$ in a single function call. This is done by simply wrapping the matrix-vector product in a call to `jax.vmap`. The result is a function that computes the matrix-vector product for each vector in $X$ and returns the result as a single array. This is done in the following code snippet:

In [5]:
from jax import vmap, Array


def apply_matrix_jax(A: Array, x: Array) -> Array:
    return A @ x


# Convert the NumPy arrays to JAX arrays
A_jax = jnp.array(A)
X_jax = jnp.array(X)

vectorized_apply_matrix = vmap(apply_matrix_jax, in_axes=(None, 0))
Y_vmap = vectorized_apply_matrix(A_jax, X_jax)

print(np.allclose(Y, Y_vmap))

True


Here, we simply define our function as in the same way as Equation {eq}`matvec`, and then _vectorize_ it through a call to `jax.vmap`. The `in_axes` argument specifies which axes of the input should be mapped over. By specifying `in_axes=(None, 0)`, we are telling `jax.vmap` to map over the over the first axis of the second argument (the vectors in $X$), while leaving the first argument (the matrix $A$) unchanged. The result is a function that computes the matrix-vector product for each vector in $X$ and returns the result as a single array. 

Note also the simple interplay between JAX and NumPy, where we can use NumPy to generate the matrix $A$ and the vectors $X$, and then use JAX to compute the matrix-vector product. We can then go back again to NumPy in order to verify the result.

## Compiling Functions with `jit`

JAX also includes a function called `jax.jit` that can be used to compile and optimize a function. This can be useful when you have a function that you want to run many times, as the compiled version will typically be much faster than the uncompiled version. To use `jax.jit`, you simply wrap your function in a call to `jax.jit`. On the first call to the compiled function, JAX will compile the function and then cache the compiled version so that it can be reused on subsequent calls.

In order to demonstrate the power of `jax.jit`, we will again consider the case where we want to compute the matrix-vector product for a large number of vectors. In order to get a baseline, we firstly time the NumPy implementation. As a more realistic usecase, $A$ is now a $100 \times 100$ matrix, and we have $10 \ 000$ vectors $x_i \in \mathbb{R}^{100}$.

In [6]:
A = rng.standard_normal((100, 100))
X = rng.standard_normal((10000, 100))

%timeit apply_matrix(A, X)

%timeit apply_matrix_einsum(A, X)

5.19 ms ± 416 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
23.8 ms ± 1.79 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


As we can see, the `np.einsum` implementation is quite a bit slower than the first implementation, without performing any further optimizations. This is due to the fact that `np.einsum` is a general-purpose function that can be used to compute a wide variety of tensor contractions, and is not optimized for the specific case of a matrix-vector product.

Next, the vectorized version:

In [7]:
# Again, we convert the NumPy arrays to JAX arrays
A_jax = jnp.array(A)
X_jax = jnp.array(X)

%timeit vectorized_apply_matrix(A_jax, X_jax)

1.69 ms ± 23.3 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


We see that the vectorized JAX version is already significantly faster than the NumPy version, even without any further optimizations. However, we can do even better by compiling the function using `jax.jit`.

In [ ]:
from jax import jit

jitted_vectorized_apply_matrix = jit(vectorized_apply_matrix)

# Warm up the JIT
%timeit -n 1 -r 1 jitted_vectorized_apply_matrix(A_jax, X_jax)

# Now time the jitted version
%timeit jitted_vectorized_apply_matrix(A_jax, X_jax)

12.3 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
752 μs ± 12.7 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Simply wrapping the function in a call to `jit` results in a 2x speedup, a 5x speedup from the original NumPy implementation. The effects become even more pronounced as the functions become more complex, especially as it then becomes more difficult to manually optimize the code.

````{note}
The `jit` function can also be used as a decorator, which can be more convenient in some cases. For example, the following code snippet is equivalent to the previous one:

```python
@jit
def apply_matrix(A, x):
    return A @ x
```
````

## Automatic Differentiation with `grad`

JAX also includes a function called `jax.grad` that can be used to differentiate another Python function. This is done using reverse-mode automatic differentiation, which is a powerful technique that can be used to compute the gradient of a function with respect to its inputs.

This method is the backbone of many machine learning algorithms. For a more detailed introduction to automatic differentiation, see this article on [automatic differentiation in Neural Networks](https://mehta-rohan.com/writings/blog_posts/autodiff.html).

Typically, differentiating a function on a computer is accomplished through finite differences, where the derivative is approximated by i.e.
```{math}
    f'(x) \approx \frac{f(x + \Delta x) - f(x)}{\Delta x}.
```
This is a simple and intuitive method, but it can be computationally expensive and inaccurate, especially when the function is complex or the gradient is high-dimensional.

As a _very_ simple example, consider the function
```{math}
    f(x) = x^2.
```

In [22]:
from typing import Callable


def f(x: Array) -> Array:
    return x**2


def diff_f(x: Array) -> Array:
    return 2 * x


def numerical_derivative(
    f: Callable[[Array], Array], x: Array, dx: float = 1e-3
) -> Array:
    return (f(x + dx) - f(x)) / dx


x = jnp.arange(0, 2.5, 0.5)

df_dx_approx = numerical_derivative(f, x)
df_dx_true = diff_f(x)

print(df_dx_approx)
print(df_dx_true)

[1.0000000e-03 1.0010004e+00 2.0010471e+00 3.0012128e+00 4.0006638e+00]
[0. 1. 2. 3. 4.]


As we see, we introduce an error with the finite difference method. Analytically, this is expected, as
```{math}
\begin{align*}
    \frac{f(x + \Delta x) - f(x)}{\Delta x} &= \frac{(x + \Delta x)^2 - x^2}{\Delta x} \\
    &= \frac{x^2 + 2x\Delta x + \Delta x^2 - x^2}{\Delta x} \\
    &= 2x + \Delta x,
\end{align*}
```
differing from the true derivative $f'(x) = 2x$ by $\Delta x$.

```{note}
The observed error is affected by numerical precision as well. There are of course more advanced methods for approximating the derivative, but these often come at an additional cost, either in terms of computational- or implementation complexity.

This is simply an illustrative example.
```

In JAX, we can instead use `jax.grad` to compute the gradient of a function. This is done by simply wrapping the function in a call to `jax.grad`. The result is a function that computes the gradient of the original function with respect to its inputs.

In [25]:
from jax import grad

automatic_derivative = vmap(grad(f))

df_dx_auto = automatic_derivative(x)
print(df_dx_auto)

[0. 1. 2. 3. 4.]


With this method, we can compute the derivative exactly, atleast in this simple case.

We have to wrap the expression in `vmap`, as `grad` only works on scalar functions. Without it, we would in this case get an error. JAX has a number of other functions for computing gradients, including `jax.jacfwd` and `jax.jacrev`, which can be used to compute the Jacobian of a function with respect to its inputs.

See the [JAX documentation](https://jax.readthedocs.io/en/latest/notebooks/autodiff_cookbook.html) for more information on automatic differentiation in JAX.